In [1]:
# Import required libraries for data manipulation and analysis
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import time
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform
from scipy.stats import spearmanr

In [2]:
#Import required sklearn functions
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import VarianceThreshold
from collections import defaultdict

In [3]:
#Import sklearn classifiers
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

In [4]:
#Import library to oversample 
from imblearn.over_sampling import RandomOverSampler

In [5]:
#Import RDKit and Mordred libraries
from rdkit import Chem
from rdkit.Chem import Draw
from mordred import Calculator, descriptors

In [6]:
#Allows figures to be visualized in jupyter notebook
%matplotlib inline

In [7]:
#Functions used in the study

#Remove those numbers from analysis data
def filter_rows_by_values1(df, col, values):
    return df[~df[col].isin(values)]

#Remove those numbers from analysis data
def filter_rows_by_values2(df, col, values):
    return df[df[col].isin(values)]

#Get Mordred calcs
def get_Mordred(data_input):
    # Assigns Reactants Mordred Info
    reactants = data_input['Substrate']
    
    reactants_mol_list = []
    for inChi_reactants in reactants:
      reactants_mol = Chem.MolFromInchi(inChi_reactants)
      reactants_mol_list.append(reactants_mol)

    # Puts reactants into Pandas Type
    reactant_data = []
    reactant_data = calc.pandas(reactants_mol_list)
       
    #Joins Mordred parameters with experimental, atomic charges, and JChem for Excel parameters
    add_reactants = pd.concat((data_input, reactant_data), axis=1)
    
    #Force any non-numeric entries as NaN and replace them with 0
    int_data = add_reactants.apply(pd.to_numeric, errors='coerce')
    
    output = int_data.fillna(0)#, inplace=True)

    return output

#Remove zero varience
def remove_zero_varience(values):
   sel = VarianceThreshold()
   _ = sel.fit(values)
   mask = sel.get_support()
   values = values.loc[:,mask] 
   return values

def remove_95correlated(correlated):
    #Remove any features that are greater than 95% correlated
    corr_matrix = correlated.corr()
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(np.bool))

    to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]

    correlated = correlated.drop(to_drop, axis = 1)
    corr_matrix = correlated.corr()
    return correlated

def remove_nonimportant(X_values, y_values):
    # Specifys Random Forest and the Number of Trees, SelectFromModel will
    # select features which are most important
    feature_names = [f"feature {i}" for i in range(X_values.shape[1])]
    forest = RandomForestClassifier(random_state=42)
    forest.fit(X_values, y_values)

    start_time = time.time()
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    elapsed_time = time.time() - start_time

    threshold = np.sort(importances)[-100]
    
    sel = SelectFromModel(RandomForestClassifier(n_estimators = 800, max_depth=30),threshold=threshold)
    sel.fit(X_values, y_values)

    # Select the final features set 
    sel.get_support()
    selected_feat= X_values.columns[(sel.get_support())]

    # Prints the names of the final selected features
    print(selected_feat)
    X_values = X_values[selected_feat]
    
    return X_values

def dendrogram(X_values, y):
    corr = spearmanr(X_values).correlation
    # Ensure the correlation matrix is symmetric
    corr = (corr + corr.T) / 2
    np.fill_diagonal(corr, 1)
    distance_matrix = 1 - np.abs(corr)
    dist_linkage = hierarchy.ward(squareform(distance_matrix))
  
    trained_cluster_ids = hierarchy.fcluster(dist_linkage, y, criterion="distance")
    trained_cluster_id_to_feature_ids = defaultdict(list) 
    for idx, trained_cluster_id in enumerate(trained_cluster_ids):
        trained_cluster_id_to_feature_ids[trained_cluster_id].append(idx)
    
    trained_selected_features = [v[0] for v in trained_cluster_id_to_feature_ids.values()]
    final_selected_features = X_values.columns[trained_selected_features]
    X_train = X_values[final_selected_features]
    return X_train

def classificationMetrics(results, y_test, pred):
    acc = accuracy_score(y_test, pred)
    prec = precision_score(y_test, pred, average=None, zero_division=0)
    recall = recall_score(y_test, pred, average=None)
    F1 = f1_score(y_test, pred, average=None)
    comb = np.concatenate((x, y, acc, prec, recall, F1), axis=None)
    comb = [comb]
    results = results.append(pd.DataFrame(comb, columns=results.columns), ignore_index=True)
    return results


In [8]:
# Sets Pandas Display to Monitor Code
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 200)

In [9]:
# Create Mordred Calculator
calc = Calculator(descriptors, ignore_3D=True)

In [10]:
# Read Training/Test data input File
data = pd.read_csv('BorylationTrainingTest 1-10-25.csv')
data.head()

,Substrate,Product,Boronic Ester,Active Catalyst-Ligand,Catalyst,Ligand,Buried_Vol,PyramidalizationAR,PyramidalizationG,SASA_area,SASA_vol,Sterimol_L,SterimolB_1,Sterimol_B_5,Buried_Sterimol_L,Buried_SterimolB_1,Buried_Sterimol_B_5,Hirshfeld Heavy Atom Charge,CM5 Charge,Hirshfeld Carbon Charge,Hirshfeld Hydrogen Charge,ESP Heavy Atom Charge,ESP Carbon Charge,ESP Hydrogen Charge,NPA Carbon Charge,NPA Hydrogen Charge,MBS Heavy Atom Charge,MBS Carbon Charge,MBS Hydrogen Charge,Mulliken Heavy Charge,Mulliken Carbon Charge,Mulliken Hydrogen Charge,Solvent,Temp,Aliphatic Atom Count,Aliphatic Bond Count,Aliphatic Ring Count,Aromatic Atom Count,Aromatic Bond Count,Aromatic Ring Count,Steric Effect Index,Atomic_Polarizability,Balaban Index,Chain Atom Count,Distance Degree,Dreiding Energy,Eccentricity,Harary Index,Hydrogen Acceptor Count,Hydrogen Acceptor Site Count,Heteroatom Aromatic Ring Count,Hydrogen Donor Count,Hydrogen Donor Site Count,Hyper Wiener Index,Largest Ring Size,Max Projection Area,Max Projection Radius,MaxZ,Moleculare Polarizability,Platt Index,Refractivity,Ring Atom Count,Rot Bond Count,Sigma Electronegativity,Wiener Index,Product_Ratio
0,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",InChI=1S/C14H29BO2/c1-6-7-8-9-10-11-12-15-16-1...,2,6,1,1,43.1,0.683535,10.223844,735.602381,1441.376195,5.941608,4.790830,7.558608,5.941608,4.790830,7.278100,-0.003389,-0.000578,-0.101759,0.032223,-0.078389,-0.182602,0.032537,-0.57256,0.19272,-0.008908,-0.254190,0.080765,-0.220320,-0.621212,0.132089,2,150,8,7,0,0,0,0,1.193207,1.116291,2.530060,8,28,12.13,7,13.742857,0,0,0,0,0,210,0,52.554143,6.587896,5.713901,15.522910,12,38.6102,0,5,7.387931,84,1
1,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",InChI=1S/C14H29BO2/c1-7-8-9-10-11-12(2)15-16-1...,2,6,1,1,43.1,0.683535,10.223844,735.602381,1441.376195,5.941608,4.790830,7.558608,5.941608,4.790830,7.278100,0.003094,0.000249,-0.057714,0.030404,0.098246,0.182934,-0.042344,-0.38255,0.18757,0.009321,-0.138428,0.073874,0.044556,-0.224805,0.134680,2,150,8,7,0,0,0,0,2.190292,1.116291,2.530060,8,22,12.13,6,13.742857,0,0,0,0,0,210,0,52.554143,6.587896,5.713901,15.522910,12,38.6102,0,5,7.470532,84,0
2,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",InChI=1S/C14H29BO2/c1-7-9-10-11-12(8-2)15-16-1...,2,6,1,1,43.1,0.683535,10.223844,735.602381,1441.376195,5.941608,4.790830,7.558608,5.941608,4.790830,7.278100,-0.000052,-0.000032,-0.059257,0.029602,-0.011526,0.182934,-0.012016,-0.37899,0.18768,-0.000222,-0.141245,0.070512,0.012779,-0.261495,0.137137,2,150,8,7,0,0,0,0,2.310662,1.116291,2.530060,8,18,12.13,5,13.742857,0,0,0,0,0,210,0,52.554143,6.587896,5.713901,15.522910,12,38.6102,0,5,7.493894,84,0
3,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",InChI=1S/C14H29BO2/c1-7-9-11-12(10-8-2)15-16-1...,2,6,1,1,43.1,0.683535,10.223844,735.602381,1441.376195,5.941608,4.790830,7.558608,5.941608,4.790830,7.278100,0.000299,0.000312,-0.058954,0.029626,-0.008331,0.037009,-0.022670,-0.37717,0.18811,-0.000191,-0.141973,0.070891,0.162986,-0.115385,0.139186,2,150,8,7,0,0,0,0,2.339699,1.116291,2.530060,8,16,12.13,4,13.742857,0,0,0,0,0,210,0,52.554143,6.587896,5.713901,15.522910,12,38.6102,0,5,7.496149,84,0
4,"InChI=1S/C7H15N/c1-2-8-6-4-3-5-7-8/h2-7H2,1H3","InChI=1S/C13H26BNO2/c1-12(2)13(3,4)17-14(16-12...",2,7,3,1,41.0,1.059976,9.242505,814.372698,1707.085805,6.090976,5.055852,9.142000,6.090976,5.055852,7.624077,0.002028,0.013777,-0.102309,0.036494,-0.043709,-0.156454,0.023226,-0.58186,0.20119,0.006751,-0.255789,0.085570,-0.086611,-0.497717,0.137539,3,150,8,8,1,0,0,0,2.188081,1.116291,2.125016,2,16,21.20,4,15.783333,1,1,0,0,0,122,6,43.497048,4.404837,6.112425,14.445701,18,36.8787,6,1,7.937482,64,1


In [11]:
#group the compounds by numbers
data['grouped'] = data.groupby('Substrate', sort=False).ngroup()
data[['Substrate','grouped']]

,Substrate,grouped
0,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",0
1,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",0
2,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",0
3,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",0
4,"InChI=1S/C7H15N/c1-2-8-6-4-3-5-7-8/h2-7H2,1H3",1
...,...,...
966,InChI=1S/C15H18/c1-10(2)13-7-5-11(3)14-8-6-12(...,188
967,InChI=1S/C15H18/c1-10(2)13-7-5-11(3)14-8-6-12(...,188
968,InChI=1S/C15H18/c1-10(2)13-7-5-11(3)14-8-6-12(...,188
969,InChI=1S/C15H18/c1-10(2)13-7-5-11(3)14-8-6-12(...,188


In [12]:
#Convert substrates to Mordred features
data = get_Mordred(data)

100%|██████████| 971/971 [01:21<00:00, 11.90it/s]


In [13]:
DTResults_df = pd.DataFrame(columns =  ['x', 'y', 'acc', 'precision 0',
                                   'precision 1','recall 0', 'recall 1', 'F1 0', 'F1 1'])

NBResults_df = pd.DataFrame(columns =  ['x', 'y', 'acc', 'precision 0',
                                   'precision 1','recall 0', 'recall 1', 'F1 0', 'F1 1'])

DTmaxacc_comb = pd.DataFrame()
NBmaxacc_comb = pd.DataFrame()

model_columns = pd.DataFrame()
for_range = range(1, 11)
for x in for_range:
    #Get numbers to represent compounds
    arr = np.arange(0, 188,  dtype=int)

    #Get 20% of numbers, without replacement
    set_numbers = np.random.choice(arr, int(len(arr)*0.20), replace=False ) 
    
    #Seperate training (80%) and test data (20%)
    training_data = filter_rows_by_values1(data, "grouped", set_numbers)
    training_data = training_data.drop('grouped', axis = 1)    
    test_data = filter_rows_by_values2(data, "grouped", set_numbers)
    test_data = test_data.drop('grouped', axis = 1)     
     
    #Remove features that dont change
    training_data = remove_zero_varience(training_data)
    
    #Remove features that are more than 95% correlated
    training_data = remove_95correlated(training_data)
    
    # Seperate dataset as response variable (Product Ratio) and feature variables
    #Note: Product Ratio is described as "0" for non-borylating sites and "1" for borylating sites
    training_X = training_data.drop('Product_Ratio' , axis = 1)
    training_y = training_data['Product_Ratio']
    test_X = test_data.drop('Product_Ratio' , axis = 1)
    test_y = test_data['Product_Ratio']
    
    #Remove features that are considered less important
    feature_names = [f"feature {i}" for i in range(training_X.shape[1])]
    forest = RandomForestClassifier(random_state=42)
    forest.fit(training_X, training_y)
    
    start_time = time.time()
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    elapsed_time = time.time() - start_time
    
    threshold = np.sort(importances)[-100] 
    sel = SelectFromModel(RandomForestClassifier(n_estimators = 800, max_depth=30),threshold=threshold)
    sel.fit(training_X, training_y)
     
    # Select the reduced features set 
    sel.get_support()
    selected_feat= training_X.columns[(sel.get_support())]
    
    reduced1_X = training_X[selected_feat]
    test_X = test_X[selected_feat]
    
    #Apply over-sampling to dataset
    ros = RandomOverSampler(random_state=10)
    X_resampled, y_resampled = ros.fit_resample(reduced1_X, training_y) 
    
    for y in [ 
              0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35, 0.35,
              0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 0.40, 
              0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45,
              0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50
             ]:
    
        #Make final training and test set and save them as df's  
        X_train = dendrogram(X_resampled, y)
        test_X = test_X[X_train.columns]
        training_columns_list = X_train.columns.tolist()
        training_columns_list = (x, y, training_columns_list)
        training_columns_list = (pd.DataFrame(training_columns_list).T)
                                        
        #Decision Tree CLassifier
        DT = DecisionTreeClassifier(max_depth=9)
        DT.fit(X_train, y_resampled)
        pred_DT = DT.predict(test_X)
        DTResults_df = classificationMetrics(DTResults_df, test_y, pred_DT)
 
        #Determine the mean accuracy of the different dendrogram settings
        acc_mean = DTResults_df.groupby('y')['acc'].mean()
        acc_std = DTResults_df.groupby('y')['acc'].std()
        precision_0_mean = DTResults_df.groupby('y')['precision 0'].mean()
        precision_0_std = DTResults_df.groupby('y')['precision 0'].std()        
        precision_1_mean = DTResults_df.groupby('y')['precision 1'].mean()
        precision_1_std = DTResults_df.groupby('y')['precision 1'].std()
        recall_0_mean = DTResults_df.groupby('y')['recall 0'].mean()
        recall_0_std = DTResults_df.groupby('y')['recall 0'].std()      
        recall_1_mean = DTResults_df.groupby('y')['recall 1'].mean()
        recall_1_std = DTResults_df.groupby('y')['recall 1'].std()
        F1_0_mean = DTResults_df.groupby('y')['F1 0'].mean()
        F1_0_std = DTResults_df.groupby('y')['F1 0'].std()
        F1_1_mean = DTResults_df.groupby('y')['F1 1'].mean()
        F1_1_std = DTResults_df.groupby('y')['F1 1'].std()
        
        DTaverage_df = pd.concat([acc_mean , acc_std, 
                                   precision_0_mean, precision_0_std, 
                                   precision_1_mean, precision_1_std, 
                                   recall_0_mean, recall_0_std, 
                                   recall_1_mean, recall_1_std,
                                   F1_0_mean, F1_0_std,
                                   F1_1_mean, F1_1_std], axis=1)

        DTaverage_df.columns = ['acc_mean' , 'acc_std', 'precision_0_mean', 'precision_0_std', 
                                 'precision_1_mean', 'precision_1_std', 'recall_0_mean', 'recall_0_std', 
                                 'recall_1_mean','recall_1_std', 'F1_0_mean', 'F1_0_std', 
                                 'F1_1_mean', 'F1_1_std']                                 
          
        
                
        DTmaxacc = DTaverage_df[DTaverage_df.acc_mean == DTaverage_df.acc_mean.max()]
        DTmaxacc_copy  = DTmaxacc.copy()
        DTmaxacc_copy['x_col'] = x       
        
        #Naive Bayes CLassifier
        NB = GaussianNB()
        NB.fit(X_train, y_resampled)
        pred_NB = NB.predict(test_X)
        NBResults_df = classificationMetrics(NBResults_df, test_y, pred_NB)
 
            #Determine the mean accuracy of the different dendrogram settings
        acc_mean = NBResults_df.groupby('y')['acc'].mean()
        acc_std = NBResults_df.groupby('y')['acc'].std()
        precision_0_mean = NBResults_df.groupby('y')['precision 0'].mean()
        precision_0_std = NBResults_df.groupby('y')['precision 0'].std()        
        precision_1_mean = NBResults_df.groupby('y')['precision 1'].mean()
        precision_1_std = NBResults_df.groupby('y')['precision 1'].std()
        recall_0_mean = NBResults_df.groupby('y')['recall 0'].mean()
        recall_0_std = NBResults_df.groupby('y')['recall 0'].std()      
        recall_1_mean = NBResults_df.groupby('y')['recall 1'].mean()
        recall_1_std = NBResults_df.groupby('y')['recall 1'].std()
        F1_0_mean = NBResults_df.groupby('y')['F1 0'].mean()
        F1_0_std = NBResults_df.groupby('y')['F1 0'].std()
        F1_1_mean = NBResults_df.groupby('y')['F1 1'].mean()
        F1_1_std = NBResults_df.groupby('y')['F1 1'].std()
        
        NBaverage_df = pd.concat([acc_mean , acc_std, 
                                   precision_0_mean, precision_0_std, 
                                   precision_1_mean, precision_1_std, 
                                   recall_0_mean, recall_0_std, 
                                   recall_1_mean, recall_1_std,
                                   F1_0_mean, F1_0_std,
                                   F1_1_mean, F1_1_std], axis=1)

        NBaverage_df.columns = ['acc_mean' , 'acc_std', 'precision_0_mean', 'precision_0_std', 
                                 'precision_1_mean', 'precision_1_std', 'recall_0_mean', 'recall_0_std', 
                                 'recall_1_mean','recall_1_std', 'F1_0_mean', 'F1_0_std', 
                                 'F1_1_mean', 'F1_1_std']                                 

        NBmaxacc = NBaverage_df[NBaverage_df.acc_mean == NBaverage_df.acc_mean.max()]
        NBmaxacc_copy  = NBmaxacc.copy()
        NBmaxacc_copy['x_col'] = x

    DTmaxacc_comb = DTmaxacc_comb.append(DTmaxacc_copy)
    NBmaxacc_comb = NBmaxacc_comb.append(NBmaxacc_copy)

#Write the results onto a CSV file, currently commented out 
DTmaxacc_comb.to_csv("10Runs_ClassificationScreening_DT_NB.csv",)
NBmaxacc_comb.to_csv("10Runs_ClassificationScreening_DT_NB.csv", mode="a")


In [14]:
DTmaxacc_comb

,acc_mean,acc_std,precision_0_mean,precision_0_std,precision_1_mean,precision_1_std,recall_0_mean,recall_0_std,recall_1_mean,recall_1_std,F1_0_mean,F1_0_std,F1_1_mean,F1_1_std,x_col
y,,,,,,,,,,,,,,,
0.35,0.765714,0.009331,0.873078,0.005293,0.455769,0.017195,0.822464,0.009198,0.554054,0.019111,0.846998,0.006529,0.500044,0.016844,1
0.35,0.772219,0.011317,0.888851,0.017201,0.456193,0.016270,0.816199,0.011507,0.602703,0.054831,0.850811,0.007531,0.518380,0.025351,2
0.35,0.774669,0.011498,0.893161,0.015460,0.458019,0.016644,0.815273,0.011597,0.616216,0.049378,0.852304,0.007810,0.524718,0.024118,3
0.35,0.754671,0.036900,0.880346,0.026504,0.432996,0.047055,0.798955,0.030898,0.586486,0.069299,0.837538,0.027087,0.497564,0.053199,4
0.35,0.749010,0.035124,0.891902,0.033378,0.422914,0.046957,0.781481,0.044993,0.627568,0.104074,0.831884,0.026936,0.501225,0.048506,5
0.35,0.749505,0.032389,0.888902,0.031231,0.423986,0.043584,0.784925,0.042043,0.616667,0.098206,0.832646,0.024857,0.498810,0.045103,6
0.35,0.754167,0.032283,0.895389,0.033102,0.428319,0.042038,0.785192,0.039081,0.637452,0.104675,0.835724,0.024348,0.508861,0.048879,7
0.35,0.753926,0.030238,0.896730,0.031157,0.424043,0.040999,0.784146,0.036683,0.638176,0.097920,0.835820,0.022799,0.506345,0.046268,8
0.35,0.750081,0.030779,0.891420,0.033067,0.418069,0.042620,0.784149,0.034996,0.620120,0.105912,0.833527,0.022669,0.496316,0.052390,9


In [15]:
NBmaxacc_comb

,acc_mean,acc_std,precision_0_mean,precision_0_std,precision_1_mean,precision_1_std,recall_0_mean,recall_0_std,recall_1_mean,recall_1_std,F1_0_mean,F1_0_std,F1_1_mean,F1_1_std,x_col
y,,,,,,,,,,,,,,,
0.35,0.571429,0.000000,0.888889,0.000000,0.297872,0.000000,0.521739,0.000000,0.756757,0.000000,0.657534,0.000000,0.427481,0.000000,1
0.35,0.575608,0.004288,0.903747,0.015244,0.295995,0.001926,0.522459,0.000739,0.783784,0.027729,0.662100,0.004685,0.429568,0.002141,2
0.35,0.582663,0.010726,0.900370,0.013260,0.295156,0.001971,0.536225,0.019811,0.765766,0.034285,0.671853,0.014531,0.425913,0.005535,3
0.45,0.578968,0.023036,0.898573,0.012028,0.298872,0.013479,0.528891,0.033090,0.770270,0.030602,0.665299,0.026567,0.430436,0.016219,4
0.45,0.590537,0.031124,0.895329,0.012574,0.295195,0.014134,0.551162,0.053814,0.740541,0.065977,0.680649,0.039032,0.421660,0.022886,5
0.45,0.590985,0.028382,0.892404,0.013222,0.295804,0.012954,0.553349,0.049289,0.734234,0.061785,0.681700,0.035650,0.421275,0.020874,6
0.50,0.584480,0.035636,0.894638,0.017460,0.290263,0.016029,0.544466,0.063462,0.737452,0.083226,0.674359,0.045746,0.415736,0.025791,7
0.45,0.576745,0.034904,0.890657,0.019674,0.282654,0.026359,0.538166,0.051476,0.729730,0.071958,0.669288,0.038056,0.406629,0.034068,8
0.45,0.578319,0.033188,0.891041,0.018568,0.285436,0.026064,0.539121,0.048574,0.732733,0.068331,0.670320,0.035974,0.410059,0.033547,9
